1. PARSE XML FILES
2. TURN INTO TREE
3. IF FAILS, PARSE AS MANY OF THE ENTRIES AS POSSIBLE

In [4]:
import traceback
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import shutil
import os

import git

repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)

srcpaths=(
    "/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls",
)

xmls = []
for d in srcpaths:
    l = list(Path(d).glob("*.xml"))
    l = [p for p in l if ".repair." not in p.name]
    xmls += l
print(len(xmls), "XML files found,", len(set([f.name for f in xmls])), "unique. duplicates =", [f for f in xmls if [g.name for g in xmls].count(f.name) > 1])

178 XML files found, 178 unique. duplicates = []


In [5]:
dstdir=Path("postprocessed_xmls")

In [19]:
if dstdir.exists():
    shutil.rmtree(dstdir)
dstdir.mkdir(exist_ok=True)

In [20]:
import tqdm as tqdm

successes_1 = []
failed_1 = []
for fpath in tqdm.tqdm(xmls):
    try:
        it = ET.iterparse(fpath)
        for tag in it:
            pass
        successes_1.append(fpath)
        shutil.copyfile(fpath, dstdir/fpath.name)
    except ET.ParseError as ex:
        failed_1.append(fpath)

print("ROUND 1:", len(failed_1), "files failed parsing")
(dstdir/"1_repair_success_1.txt").write_text("\n".join(map(str, successes_1)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 178/178 [1:20:47<00:00, 27.24s/it]


ROUND 1: 150 files failed parsing


4367

In [10]:
import tqdm as tqdm

fuzzer_start = re.compile(r'''<call[^>]*method="[^("]+.fuzzerTestOneInput\([^)]+\)"[^>]*''')

from xml.dom import minidom
from xml.parsers.expat import ExpatError

def recover_functions_simple(fpath, dstdir):
    repair_path = dstdir/(str(fpath.name) + ".repair.xml")
    if repair_path.exists():
        return repair_path
    
#     with open(fpath) as f:
#         num_lines = sum(1 for line in tqdm.tqdm(f, desc=f"count lines ({fpath.name})"))
    last_fuzzer_start = None
    with open(fpath) as f:
        i = 0
        it = iter(f)
        failed_functions = 0
        all_functions = 0
        with tqdm.tqdm(it, total=None, desc=f"chop final fuzzer target call ({fpath.name})") as pbar:
            it = pbar
            fuzz_functions = []
            current_fuzz_function = None
            for i, line in enumerate(it):
                m = fuzzer_start.search(line)
                if m is not None:
#                     print("MATCH", m, m.start(), m.span())
                    j = m.start()
                    last_fuzzer_start = (i, j)
#     print("LFS", last_fuzzer_start)
    if last_fuzzer_start is None:
        print("ERROR: last_fuzzer_start is None", fpath)
        return None
    
    with open(fpath) as f, open(repair_path, "w") as outf:
        it = iter(f)
        with tqdm.tqdm(it, total=last_fuzzer_start[0], desc=f"write file ({fpath.name})") as pbar:
            it = pbar
            for i, line in enumerate(it):
#                 print(i)
                if i == last_fuzzer_start[0]:
#                     print("LINE:")
#                     print(line)
                    line_chopped = line[:last_fuzzer_start[1]]
#                     print("CHOPPED:")
#                     print(line_chopped)
                    outf.write(line_chopped)
                    break
                else:
                    outf.write(line)
            outf.write("</trace>")
        
    return repair_path

xml_file = Path("/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls")/"trace-apache-commons-cli-ParserFuzzer.xml"
recover_functions_simple(xml_file, Path("postprocessed_xmls_debug"))

PosixPath('postprocessed_xmls_debug/trace-apache-commons-cli-ParserFuzzer.xml.repair.xml')

In [11]:
fpath = Path("/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls")/"trace-apache-commons-cli-ParserFuzzer.xml"
repair_path = recover_functions_simple(fpath, Path("postprocessed_xmls_debug"))
print(fpath, "->", repair_path)
it = ET.iterparse(repair_path)
for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
    pass

/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-cli-ParserFuzzer.xml -> postprocessed_xmls_debug/trace-apache-commons-cli-ParserFuzzer.xml.repair.xml


check XML parse (trace-apache-commons-cli-ParserFuzzer.xml): 3530913it [00:31, 113565.70it/s]


ParseError: mismatched tag: line 3751875, column 2 (<string>)

In [12]:
successes_1 = list(map(Path, (dstdir/"1_repair_success_1.txt").read_text().split("\n")))
failed_1 = [fpath for fpath in xmls if fpath.name not in [s.name for s in successes_1]]
len(successes_1), len(failed_1)

(28, 150)

In [14]:
import tqdm
successes_2 = []
failed_2 = []
for fpath in tqdm.tqdm(failed_1, position=1, desc="round 2"):
    repair_path = recover_functions_simple(fpath, dstdir)
    if repair_path is None:
        print("ERROR", fpath)
        continue
    try:
        it = ET.iterparse(repair_path)
        for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
            pass
        successes_2.append(repair_path)
        # shutil.copyfile(repair_path, dstdir/repair_path.name)
    except ET.ParseError as ex:
        print("exception", type(ex), ex, "parsing", fpath, repair_path)
        failed_2.append(repair_path)

print("ROUND 2:", len(failed_2), "files failed parsing")
(dstdir/"1_repair_success_2.txt").write_text("\n".join(map(str, successes_2)))


check XML parse (trace-jackson-dataformat-xml-XmlFuzzer.xml): 1it [00:00, 1807.89it/s]                                                                                                                                | 0/150 [00:00<?, ?it/s]
check XML parse (trace-apache-commons-bcel-BcelFuzzer.xml): 81623it [00:00, 173082.51it/s]

check XML parse (trace-apache-commons-beanutils-BeanutilsFuzzer.xml): 5569750it [00:48, 115179.30it/s]                                                                                                        | 2/150 [00:00<00:35,  4.17it/s]

check XML parse (trace-apache-commons-cli-ParserFuzzer.xml): 3530913it [00:33, 106461.83it/s]                                                                                                                 | 3/150 [00:48<49:42, 20.29s/it]

round 2:   3%|█████                                                                                                                                                                         

exception <class 'xml.etree.ElementTree.ParseError'> mismatched tag: line 3751875, column 2 parsing /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-cli-ParserFuzzer.xml postprocessed_xmls/trace-apache-commons-cli-ParserFuzzer.xml.repair.xml


check XML parse (trace-apache-commons-codec-BinaryEncodingFuzzer.xml): 3796845it [00:33, 113784.95it/s]

check XML parse (trace-apache-commons-collections-CollectionsBidiMapFuzzer.xml): 1it [00:01,  1.18s/it]                                                                                                     | 5/150 [01:55<1:07:36, 27.98s/it]

check XML parse (trace-apache-commons-CompressSevenZFuzzer.xml): 846973it [00:05, 167413.97it/s]                                                                                                              | 6/150 [01:56<45:48, 19.09s/it]

check XML parse (trace-apache-commons-CompressTarFuzzer.xml): 4873660it [00:38, 126422.80it/s]                                                                                                                | 7/150 [02:01<34:44, 14.58s/it]

round 2:   5%|██████████▏                                                                                                                                                      

exception <class 'xml.etree.ElementTree.ParseError'> mismatched tag: line 4666420, column 2 parsing /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-CompressTarFuzzer.xml postprocessed_xmls/trace-apache-commons-CompressTarFuzzer.xml.repair.xml


check XML parse (trace-apache-commons-CompressZipFuzzer.xml): 4260591it [00:38, 112029.48it/s]

check XML parse (trace-apache-commons-configuration-INIConfigurationWriteFuzzer.xml): 18380it [00:01, 12152.43it/s]                                                                                         | 9/150 [03:18<1:03:35, 27.06s/it]

round 2:   7%|████████████▋                                                                                                                                                                                  | 10/150 [03:19<44:50, 19.22s/it]

...

chop final fuzzer target call (trace-tomcat-WsPingPongFuzzer.xml): 9292it [00:00, 69459.79it/s]

round 2:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 147/150 [2:08:25<01:44, 34.76s/it]

ERROR: last_fuzzer_start is None /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-tomcat-WsPingPongFuzzer.xml
ERROR /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-tomcat-WsPingPongFuzzer.xml


chop final fuzzer target call (trace-woodstox-XmlFuzzer.xml): 20417it [00:00, 73734.63it/s]
write file (trace-woodstox-XmlFuzzer.xml): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2400.86it/s]
check XML parse (trace-woodstox-XmlFuzzer.xml): 1it [00:00, 151.16it/s]

chop final fuzzer target call (trace-xstream-XmlFuzzer.xml): 123255it [00:01, 62364.05it/s]████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 148/150 [2:08:27<00:52, 26.14s/it]
write file (trace-xstream-XmlFuzzer.xml): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10979.85it/s]
check XML parse (trace-xstream-XmlFuzzer.xml): 1it [00:00, 3930.93it/s]

chop final fuzzer target call (trace-zip4j-Zi

ROUND 2: 17 files failed parsing


8704

In [15]:
all_files = successes_1 + successes_2

In [16]:
with open(dstdir/"1_repair_success_all.txt", "w") as f:
    f.write("\n".join(map(str, all_files)))